## Merchantよってamountの絶対値あたりの価値は違うという仮説で、特徴をMain Stateあたりのリフトにする

In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [3]:
# regist_termをもたせる
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
train_test.set_index(key, inplace=True)
base = utils.read_df_pkl('../input/base_term*0*').set_index(key)[['hist_regist_term', 'new_regist_term']]
train_test = train_test.join(base)

df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.16it/s]


Mem. usage decreased to  5.58 Mb (48.2% reduction)
Mem. usage decreased to  3.18 Mb (43.8% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [71]:
from time_utils import date_add_days

hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_max_date.name = 'hist_purchase_date_max'
hist_min_date.name = 'hist_purchase_date_min'
hist_max_date = hist_max_date.to_frame()
hist_min_date = hist_min_date.to_frame()

# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:7])

hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


new_max_date = df_new.groupby(key)['purchase_date'].max()
new_min_date = df_new.groupby(key)['purchase_date'].min()
new_max_date.name = 'new_purchase_date_max'
new_min_date.name = 'new_purchase_date_min'
new_max_date = new_max_date.to_frame()
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:7])
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
df_hist = df_hist.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
df_new = df_new.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)

In [72]:
df = df_hist
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth1.shape)
print(auth0.shape)

cat1_0 = False
cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]

(26595452, 28)
(2516909, 28)


### Subsector

In [52]:
# subsectorでsampleが少ないものをまとめる
tmp = df_new['subsector_id'].value_counts().reset_index()
tmp.loc[tmp['subsector_id']<7000, 'subsector_id'] = -99
tmp.loc[tmp['subsector_id']>=7000, 'subsector_id'] = tmp.loc[tmp['subsector_id']>=7000, 'index']
subsec_map = tmp.set_index('index').to_dict()['subsector_id']

In [75]:
prefix = '218_lif'
new_df_list = [new_cat1, new_cat0]
new_fname_list = ['new_cat1', 'new_cat0']
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
lift_level = 'subsector_id'
ignore_list = list(train_test.columns) + [key, lift_level]

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(new_df_list, new_fname_list):
    
    df['subsector_id'] = df['subsector_id'].map(subsec_map)
    # new
    df['date_diff'] = (df['hist_purchase_date_max'] - df['purchase_date']).dt.days
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['month_diff'] += df['month_lag']
#     df[f'amount_per_installments'] = df[f'purchase_amount'] / (df[f'installments'] + 1.0)
    
    aggs = {}
#     col_unique =['merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    
#     for col in col_unique:
#         aggs[col] = ['nunique']

#     aggs['purchase_amount'] = ['sum']
#     aggs['installments'] = ['mean', 'max']
#     aggs['amount_per_installments'] = ['mean', 'sum']
#     aggs['month_lag'] = ['max','mean']
#     aggs['month_diff'] = ['mean','std']
#     aggs['card_id'] = ['size']
    aggs['date_diff'] = ['mean','max']
    
    if key not in df.columns:
        df.reset_index(inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby([key, lift_level]).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(inplace=True)
    
    lift_list = sorted(list(set(df_agg['subsector_id'].values)))
    feat_cols = [col for col in df_agg.columns if col not in ignore_list]
    
    df_list = []
    for level in lift_list:
        
        tmp_list = []
        for col in feat_cols:
            tmp = df_agg[df_agg[lift_level]==level].reset_index()
            tmp = tmp[[key, col]]
            base_avg = tmp[col].mean()
            tmp.set_index(key, inplace=True)
            tmp[col] /= base_avg
            tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
            tmp_list.append(tmp)
        df_level = pd.concat(tmp_list, axis=1)
        df_level[lift_level] = level
        df_list.append(df_level)
        
    df_lift = pd.concat(df_list, axis=0)
    df_lift[lift_level] =  df_lift[lift_level].fillna(-1).astype('int64')
    df_lift = df_lift.reset_index().set_index([key, lift_level])
    print(df_lift.shape)
    for tmp_col in df_lift.columns:
        try:
#             df_yoko = df_lift.reset_index().fillna(0).pivot_table(index=key, columns=lift_level, values=tmp_col).fillna(0)
            df_lift[tmp_col] = df_lift[tmp_col].astype('float32')
            df_yoko = df_lift[tmp_col].unstack().fillna(0)
        except AttributeError:
            print(tmp_col)
            display(df_lift.head())
            sys.exit()
        df_yoko.columns = [f"{lift_level}_{c}_{tmp_col}" for c in df_yoko.columns]
        print(df_yoko.shape)
        
        #========================================================================
        # Save Feature
        base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
        if target in df_yoko.columns:
            df_yoko.drop(target, axis=1, inplace=True)
        df_yoko = base.join(df_yoko)
        for col in df_yoko.columns:
            print(df_yoko.shape)
            if col in ignore_list: continue
            if col.count('feature_'):continue
            if not(col.count('new_cat')):continue
            feature = df_yoko[col].fillna(0).astype('float32').values
            col = col.replace('.', '_')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
            #     utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
            #========================================================================









  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 51.13it/s]

Aggregation Start!
(63096, 30)










  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 57.82it/s]

(59321, 2)
(45471, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)










  0%|          | 0/3 [00:00<?, ?it/s]

...

(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
Aggregation Start!
(1899935, 30)
(1241571, 2)










  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 57.64it/s]

(284290, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)










  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 49.84it/s]

(284290, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)


In [74]:
prefix = '218_lif'
auth_df_list = [auth1_cat1, auth1_cat0]
auth_fname_list = ['auth1_cat1', 'auth1_cat0']
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
lift_level = 'subsector_id'
ignore_list = list(train_test.columns) + [key, lift_level]

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(auth_df_list, auth_fname_list):
    
    df['subsector_id'] = df['subsector_id'].map(subsec_map)
    # hist
    df['date_diff'] = (df['hist_purchase_date_max'] - df['purchase_date']).dt.days
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df[f'amount_per_installments'] = df[f'purchase_amount'] / (df[f'installments'] + 1.0)
    
    aggs = {}
    col_unique =['merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    
    for col in col_unique:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum']
    aggs['installments'] = ['mean', 'max']
    aggs['amount_per_installments'] = ['mean', 'sum']
    aggs['month_lag'] = ['max','mean']
    aggs['month_diff'] = ['mean','std']
    aggs['date_diff'] = ['mean','max']
    aggs['card_id'] = ['size']
    
    if key not in df.columns:
        df.reset_index(inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby([key, lift_level]).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(inplace=True)
    
    lift_list = sorted(list(set(df_agg[lift_level].values)))
    feat_cols = [col for col in df_agg.columns if col not in ignore_list]
    
    df_list = []
    for level in lift_list:
        
        tmp_list = []
        for col in feat_cols:
            tmp = df_agg[df_agg[lift_level]==level].reset_index()
            tmp = tmp[[key, col]]
            base_avg = tmp[col].mean()
            tmp.set_index(key, inplace=True)
            tmp[col] /= base_avg
            tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
            tmp_list.append(tmp)
        df_level = pd.concat(tmp_list, axis=1)
        df_level[lift_level] = level
        df_list.append(df_level)
        
    df_lift = pd.concat(df_list, axis=0)
    df_lift[lift_level] =  df_lift[lift_level].fillna(-1).astype('int64')
    df_lift = df_lift.reset_index().set_index([key, lift_level])
    print(df_lift.shape)
    for tmp_col in df_lift.columns:
        try:
#             df_yoko = df_lift.reset_index().fillna(0).pivot_table(index=key, columns=lift_level, values=tmp_col).fillna(0)
            df_lift[tmp_col] = df_lift[tmp_col].astype('float32')
            df_yoko = df_lift[tmp_col].unstack().fillna(0)
        except AttributeError:
            print(tmp_col)
            display(df_lift.head())
            sys.exit()
        df_yoko.columns = [f"{lift_level}_{c}_{tmp_col}" for c in df_yoko.columns]
        print(df_yoko.shape)
        
        #========================================================================
        # Save Feature
        base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
        if target in df_yoko.columns:
            df_yoko.drop(target, axis=1, inplace=True)
        df_yoko = base.join(df_yoko)
        for col in df_yoko.columns:
            print(df_yoko.shape)
            if col in ignore_list: continue
            if col.count('feature_'):continue
            if not(col.count('auth1_cat')):continue
            feature = df_yoko[col].fillna(0).astype('float32').values
            col = col.replace('.', '_')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
            #     utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
            #========================================================================









  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 77.06it/s]

Aggregation Start!
(1560059, 32)










  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 75.29it/s]

(300342, 16)
(121357, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)










  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 69.09it/s]

...

(320220, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)









  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 49.13it/s]


(320220, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)










  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 28.70it/s]









(320220, 29)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
(325540, 30)
